# 天气预报微调

## 加载1.8B Chat Int4模型
在原有chat模型上进行参数微调

In [ ]:
from modelscope import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer

In [27]:
model_dir = snapshot_download('qwen/Qwen-1_8B-Chat-Int4')

tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True
).eval()

2024-02-27 08:26:49,428 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
Try importing flash-attention for faster inference...


In [33]:
prompt = '{\'id\': \'identity_1\', \'conversations\': [{\'from\': \'user\', \'value\': \'\\n给定一句话：“你们1999年4月8日去金溪县玩吗？”，请你按步骤要求工作。\\n\\n步骤1：识别这句话中的城市和日期共2个信息\\n步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}\\n\\n请问，这个JSON字符串是：\\n\'}, {\'from\': \'assistant\', \'value\': \'{"city": "金溪县", "date": "1999-04-08"}\'}]}'


In [36]:
response, _ = model.chat(tokenizer
                         , prompt
                         , history=None
                        )
print(response)

从给定的对话中可以看出，这句话是在询问一个人是否在1999年4月8日去了金溪县玩。以下是如何使用Python来完成这个任务的：

```python
def extract_info(text):
    # 识别句子中的城市和日期信息
    city = text.split('-')[1]
    date_str = text.split('，")[1]

    # 根据城市和日期信息，生成JSON字符串
    city_dict = {"city": city, "date": date_str}
    return json.dumps(city_dict)

text = """
给你们1999年4月8日去金溪县玩吗？
"""

print(extract_info(text))
```

这段代码首先定义了一个函数`extract_info`，它接收一个文本作为输入，然后通过分割句子并提取城市和日期信息来实现。最后，将城市和日期信息转换为JSON字符串并返回。

运行这段代码后，你将会得到输出："{"city": "金溪县", "date": "1999-04-08"}"，这就是符合题目要求的JSON字符串。


## 有效参数微调

In [2]:
# 城市数据
with open('city.txt','r',encoding='utf-8') as fp:
    city_list=fp.readlines()
    city_list=[line.strip().split(' ')[1] for line in city_list]

In [3]:
Q='青岛4月6日下雨么?'

prompt_template='''
给定一句话：“%s”，请你按步骤要求工作。

步骤1：识别这句话中的城市和日期共2个信息
步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}

请问，这个JSON字符串是：
'''

# 生成SFT微调数据

Qwen的SFT数据格式要求:
```
[
  {
    "id": "identity_0",
    "conversations": [
      {
        "from": "user",
        "value": "你好"
      },
      {
        "from": "assistant",
        "value": "我是一个语言模型，我叫通义千问。"
      }
    ]
  }
]
```

In [4]:
import random
import json
import time 

Q_arr=[]
A_arr=[]

Q_list=[
    ('{city}{year}年{month}月{day}日的天气','%Y-%m-%d'),
    ('{city}{year}年{month}月{day}号的天气','%Y-%m-%d'),
    ('{city}{month}月{day}日的天气','%m-%d'),
    ('{city}{month}月{day}号的天气','%m-%d'),

    ('{year}年{month}月{day}日{city}的天气','%Y-%m-%d'),
    ('{year}年{month}月{day}号{city}的天气','%Y-%m-%d'),
    ('{month}月{day}日{city}的天气','%m-%d'),
    ('{month}月{day}号{city}的天气','%m-%d'),

    ('你们{year}年{month}月{day}日去{city}玩吗？','%Y-%m-%d'),
    ('你们{year}年{month}月{day}号去{city}玩么？','%Y-%m-%d'),
    ('你们{month}月{day}日去{city}玩吗？','%m-%d'),
    ('你们{month}月{day}号去{city}玩吗？','%m-%d'),
]

# 生成一批"1月2号"、"1月2日"、"2023年1月2号", "2023年1月2日", "2023-02-02", "03-02"之类的话术, 教会它做日期转换
for i in range(1000):
    Q=Q_list[random.randint(0,len(Q_list)-1)]
    city=city_list[random.randint(0,len(city_list)-1)]
    year=random.randint(1990,2025)
    month=random.randint(1,12)
    day=random.randint(1,28)
    time_str='{}-{}-{}'.format(year,month,day)
    date_field=time.strftime(Q[1],time.strptime(time_str,'%Y-%m-%d'))
    Q=Q[0].format(city=city,year=year,month=month,day=day) # 问题
    A=json.dumps({'city':city,'date':date_field},ensure_ascii=False)  # 回答

    Q_arr.append(prompt_template%(Q,))
    A_arr.append(A)

import pandas as pd 

df=pd.DataFrame({'Prompt':Q_arr,'Completion':A_arr})
df.to_excel('train.xlsx')

In [5]:
def get_finetinu_data(idx, Q, A):
    data_tample = {
    "id": f"identity_{idx}",
    "conversations": [
    {
    "from": "user",
    "value": f"{Q}"
    },
    {
    "from": "assistant",
    "value": f"{A}"
    }
    ]
    }
    return data_tample

In [6]:
data_res = []
for idx, items in enumerate(df.to_dict(orient="records")):
    # print(idx, key)
    data = get_finetinu_data(idx, items["Prompt"], items["Completion"])
    data_res.append(data)
    # break

In [17]:
df

,Prompt,Completion
0,\n给定一句话：“你们2017年1月15日去榆阳区玩吗？”，请你按步骤要求工作。\n\n步骤...,"{""city"": ""榆阳区"", ""date"": ""2017-01-15""}"
1,\n给定一句话：“你们1999年4月8日去金溪县玩吗？”，请你按步骤要求工作。\n\n步骤1...,"{""city"": ""金溪县"", ""date"": ""1999-04-08""}"
2,\n给定一句话：“潮州市10月7号的天气”，请你按步骤要求工作。\n\n步骤1：识别这句话中...,"{""city"": ""潮州市"", ""date"": ""10-07""}"
3,\n给定一句话：“2024年8月10日舟曲县的天气”，请你按步骤要求工作。\n\n步骤1：识...,"{""city"": ""舟曲县"", ""date"": ""2024-08-10""}"
4,\n给定一句话：“你们2018年1月2号去临渭区玩么？”，请你按步骤要求工作。\n\n步骤1...,"{""city"": ""临渭区"", ""date"": ""2018-01-02""}"
...,...,...
995,\n给定一句话：“安乡县4月16号的天气”，请你按步骤要求工作。\n\n步骤1：识别这句话中...,"{""city"": ""安乡县"", ""date"": ""04-16""}"
996,\n给定一句话：“你们7月20号去长寿区玩吗？”，请你按步骤要求工作。\n\n步骤1：识别这...,"{""city"": ""长寿区"", ""date"": ""07-20""}"
997,\n给定一句话：“恩施土家族苗族自治州6月16日的天气”，请你按步骤要求工作。\n\n步骤1...,"{""city"": ""恩施土家族苗族自治州"", ""date"": ""06-16""}"
998,\n给定一句话：“7月1号梧州市的天气”，请你按步骤要求工作。\n\n步骤1：识别这句话中的...,"{""city"": ""梧州市"", ""date"": ""07-01""}"


In [7]:
import pickle

In [8]:
import json

In [9]:
with open("train.txt", "w") as fp:
    fp.write(json.dumps(data_res, ensure_ascii=False))

In [10]:
pd.Series(data_res).to_json("train_data.json", orient="records")

In [11]:
pwd

'/root/LLM/Qwen/examples/天气预报'

# 微调模型，生成到output_qwen

bash finetune/finetune_qlora_single_gpu.sh  -m /root/.cache/modelscope/hub/qwen/Qwen-1_8B-Chat-Int4 -d /root/Qwen/train.txt

# 加载SFT后的模型

In [12]:
# from transformers import A

In [13]:
# AutoTokenizer.from_pretrained("../../output_qwen/checkpoint-10/")

In [23]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    '../../output_qwen/checkpoint-10/', # path to the output directory
    device_map="auto",
    trust_remote_code=True
).eval()

Try importing flash-attention for faster inference...
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 151851. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


In [24]:
model.chat(tokenizer, f"{data_res[1]}",history=None)

('{"city": "金溪县", "date": "1999-04-08"}',
 [('{\'id\': \'identity_1\', \'conversations\': [{\'from\': \'user\', \'value\': \'\\n给定一句话：“你们1999年4月8日去金溪县玩吗？”，请你按步骤要求工作。\\n\\n步骤1：识别这句话中的城市和日期共2个信息\\n步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}\\n\\n请问，这个JSON字符串是：\\n\'}, {\'from\': \'assistant\', \'value\': \'{"city": "金溪县", "date": "1999-04-08"}\'}]}',
   '{"city": "金溪县", "date": "1999-04-08"}')])

In [15]:
data_res[1]

{'id': 'identity_1',
 'conversations': [{'from': 'user',
   'value': '\n给定一句话：“你们1999年4月8日去金溪县玩吗？”，请你按步骤要求工作。\n\n步骤1：识别这句话中的城市和日期共2个信息\n步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}\n\n请问，这个JSON字符串是：\n'},
  {'from': 'assistant', 'value': '{"city": "金溪县", "date": "1999-04-08"}'}]}

In [16]:
model.generation_config.top_p=0 # 只选择概率最高的token

Q_list=['2020年4月16号三亚下雨么？','青岛3-15号天气预报','5月6号下雪么，城市是威海','青岛2023年12月30号有雾霾么?','我打算6月1号去北京旅游，请问天气怎么样？','你们打算1月3号坐哪一趟航班去上海？','小明和小红是8月8号在上海结婚么?',
        '一起去东北看冰雕么，大概是1月15号左右，我们3个人一起']
for i, Q in enumerate(Q_list):
    prompt=f'{data_res[i]}'
    A,hist=model.chat(tokenizer,prompt,history=None)
    print('Q:%s\nA:%s\n'%(Q,A))

Q:2020年4月16号三亚下雨么？
A:{"city": "榆阳区", "date": "2017-01-15"}

Q:青岛3-15号天气预报
A:{"city": "金溪县", "date": "1999-04-08"}

Q:5月6号下雪么，城市是威海
A:{"city": "潮州市", "date": "10-07"}

Q:青岛2023年12月30号有雾霾么?
A:{"city": "舟曲县", "date": "2024-08-10"}

Q:我打算6月1号去北京旅游，请问天气怎么样？
A:{"city": "临渭区", "date": "2018-01-02"}

Q:你们打算1月3号坐哪一趟航班去上海？
A:{"city": "遂宁市", "date": "2011-12-23"}

Q:小明和小红是8月8号在上海结婚么?
A:{"city": "连州市", "date": "08-08"}

Q:一起去东北看冰雕么，大概是1月15号左右，我们3个人一起
A:{"city": "兴庆区", "date": "12-16"}



In [18]:
df.head(10)

,Prompt,Completion
0,\n给定一句话：“你们2017年1月15日去榆阳区玩吗？”，请你按步骤要求工作。\n\n步骤...,"{""city"": ""榆阳区"", ""date"": ""2017-01-15""}"
1,\n给定一句话：“你们1999年4月8日去金溪县玩吗？”，请你按步骤要求工作。\n\n步骤1...,"{""city"": ""金溪县"", ""date"": ""1999-04-08""}"
2,\n给定一句话：“潮州市10月7号的天气”，请你按步骤要求工作。\n\n步骤1：识别这句话中...,"{""city"": ""潮州市"", ""date"": ""10-07""}"
3,\n给定一句话：“2024年8月10日舟曲县的天气”，请你按步骤要求工作。\n\n步骤1：识...,"{""city"": ""舟曲县"", ""date"": ""2024-08-10""}"
4,\n给定一句话：“你们2018年1月2号去临渭区玩么？”，请你按步骤要求工作。\n\n步骤1...,"{""city"": ""临渭区"", ""date"": ""2018-01-02""}"
5,\n给定一句话：“2011年12月23日遂宁市的天气”，请你按步骤要求工作。\n\n步骤1：...,"{""city"": ""遂宁市"", ""date"": ""2011-12-23""}"
6,\n给定一句话：“你们8月8日去连州市玩吗？”，请你按步骤要求工作。\n\n步骤1：识别这句...,"{""city"": ""连州市"", ""date"": ""08-08""}"
7,\n给定一句话：“兴庆区12月16号的天气”，请你按步骤要求工作。\n\n步骤1：识别这句话...,"{""city"": ""兴庆区"", ""date"": ""12-16""}"
8,\n给定一句话：“甘州区4月7日的天气”，请你按步骤要求工作。\n\n步骤1：识别这句话中的...,"{""city"": ""甘州区"", ""date"": ""04-07""}"
9,\n给定一句话：“10月28号崆峒区的天气”，请你按步骤要求工作。\n\n步骤1：识别这句话...,"{""city"": ""崆峒区"", ""date"": ""10-28""}"


In [ ]:
prompt='青岛海边钓鱼需要特别注意什么？'
resp,hist=model.chat(tokenizer,prompt,history=None)
print(resp)